# AWS Attached Malicious Lambda Layer

Detects when an user attached a Lambda layer to an existing function to override a library that is in use by the function, where their malicious code could utilize the function's IAM role for AWS API calls.
This would give an adversary access to the privileges associated with the Lambda service role that is attached to that function.


References:
* https://docs.aws.amazon.com/lambda/latest/dg/API_UpdateFunctionConfiguration.html

False Positives:
* Lambda Layer being attached may be performed by a system administrator. Verify whether the user identity, user agent, and/or hostname should be making changes in your environment.
* Lambda Layer being attached from unfamiliar users should be investigated. If known behavior is causing false positives, it can be exempted from the rule.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
eventSource="lambda.amazonaws.com"
eventName="UpdateFunctionConfiguration*"
userIdentity.type="IAMUser"
| stats
min(@scnr.datetime) as firstTime,
max(@scnr.datetime) as lastTime,
count() as eventCount
by
userIdentity.arn,
eventSource,
eventName,
awsRegion

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName